<a href="https://colab.research.google.com/github/VigneshwaraChinnadurai/Competitions/blob/master/Cassava%20leaf%20disease%20classification/classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install pillow

In [ ]:
import keras 
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

import warnings
warnings.filterwarnings('ignore')

C:\ProgramData\Anaconda3\envs\keras_gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\ProgramData\Anaconda3\envs\keras_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\keras_gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\e

In [ ]:
model = Sequential()  
model.add(Conv2D(16, (3, 3), input_shape = (128, 128, 3),kernel_initializer='normal', activation='relu'))
model.add(Conv2D(16, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(32, (3, 3), border_mode='valid',activation='relu'))
model.add(Conv2D(32, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3, 3), border_mode='valid',activation='relu'))
model.add(Conv2D(128, (3, 3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(128, (3, 3), border_mode='valid',activation='relu'))
model.add(Conv2D(128, (3, 3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5,activation='softmax'))

ada = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
sgd = keras.optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(loss="mean_squared_error", optimizer=sgd, metrics=['accuracy'])
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(shear_range = 0.2,
                                   channel_shift_range = 0.2,
                                   zoom_range = 0.2,
                                   rotation_range=10,
                                   validation_split=0.9,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(validation_split=0.9)

In [ ]:
training_set = train_datagen.flow_from_directory('train_images',
                                                 target_size = (128, 128),
                                                 batch_size = 16,
                                                 shuffle=True,
                                                 seed=101,
                                                 #save_to_dir='Augumented/Train',
                                                 #save_format='jpeg',
                                                 interpolation='nearest',
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('eval_images',
                                            target_size = (128, 128),
                                            batch_size = 16,
                                            shuffle=True,
                                            seed=101,
                                            #save_to_dir='Augumented/Test',
                                            #save_format='jpeg',
                                            interpolation='nearest',
                                            class_mode = 'categorical')


Found 17107 images belonging to 5 classes.
Found 4282 images belonging to 5 classes.


In [ ]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=20)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', save_best_only=True)

In [ ]:
model.fit_generator(training_set,
                    steps_per_epoch = 128,
                    epochs = 100,
                    validation_data = test_set,
                    validation_steps = 70,
                    class_weight={0:10,1:5,2:5,3:1,4:5},
                    callbacks=[es,mc])

from keras.models import load_model
saved_model = load_model('best_model.h5')

from sklearn.externals import joblib
joblib.dump(model, 'Cassava leaf disease classification.joblib')  


Epoch 1/100
128/128 [==============================] - 33s 260ms/step - loss: 14.2666 - accuracy: 0.2124 - val_loss: 4.2608 - val_accuracy: 0.1080
Epoch 2/100
128/128 [==============================] - 32s 251ms/step - loss: 8.5887 - accuracy: 0.2408 - val_loss: 6.6805 - val_accuracy: 0.1143
Epoch 3/100
128/128 [==============================] - 31s 243ms/step - loss: 5.1666 - accuracy: 0.2471 - val_loss: 6.4112 - val_accuracy: 0.0821
Epoch 4/100
128/128 [==============================] - 31s 242ms/step - loss: 4.5411 - accuracy: 0.2700 - val_loss: 6.1405 - val_accuracy: 0.0709
Epoch 5/100
128/128 [==============================] - 28s 217ms/step - loss: 4.4765 - accuracy: 0.2773 - val_loss: 4.5417 - val_accuracy: 0.0571
Epoch 6/100
128/128 [==============================] - 25s 196ms/step - loss: 4.5934 - accuracy: 0.2417 - val_loss: 2.7881 - val_accuracy: 0.0589
Epoch 7/100
128/128 [==============================] - 25s 195ms/step - loss: 4.6405 - accuracy: 0.2925 - val_loss: 2.6663

C:\ProgramData\Anaconda3\envs\keras_gpu\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['Cassava leaf disease classification.joblib']

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing import image
test_image = image.load_img('test_images/2216849948.jpg', target_size = (128, 128))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = saved_model.predict(test_image)
result

array([[0.0011404 , 0.07094298, 0.4245762 , 0.46602482, 0.0373156 ]],
      dtype=float32)